# ELECTRA

> **ELECTRA: pretraining text encoders as discriminators rahter than generators**. Kevin Clark, Minh-Thang Luong, Quoc V.Le, Christopher D. Manning. ICLR 2020

### 简介

预训练模型的痛点就是：提高预训练的计算量（模型的参数多、训练语料大）必然会带来性能的提升，然而这并不是我们想要的。

如何在保证成绩不变或者更佳的前提下对模型进行轻量化（减少模型参数量、减少模型预训练的计算量），是一个值得研究的问题。

ELECTRA提出替换字符识别[^rtd]这个新的预训练任务，声称可以同时实现高效的计算和优异的性能。

本文通过一些列实验证明了替换字符识别这个预训练任务的有效性，ELECTRA在参数少的情况下与RoBERTa和XLNet成绩相当，而在参数相当的情况下ELECTRA可以超过RoBERTa和XLNet。

### 替换字符识别，replaced token detection

BERT的遮掩语言建模(MLM)预训练任务有两个诟病：（1）对于输入的每个数据样例，BERT只预测被MASK掉的token，这样就需要很多的预训练步数，非常低效；（2）BERT的预训练和微调不一致：预训练的时候输入包含[MASK]，但是微调时却没有。

如下图所示，替换字符识别预训练任务先通过一个生成器来替换输入序列中的一些token（这里的百分比和BERT的15%相同），然后判别器会预测每个token是否是被替换的。预训练结束后直接把生成器丢掉，只保留ELECTRA这个判别器在下游任务上做微调。

![image](../images/electra.png)

相比BERT的MLM，替换字符识别是在输入的所有token上学习，而不是仅限于被遮掩的token这个小的子集，这样不仅可以提高训练效率，还解决了不一致的问题。

具体地，生成器和判别器内部都是Transformer神经网络，输入一个token序列$\mathbf x = \{x_1, ..., x_n\}$，输出上下文语境表征$H = \{h_1, ..., h_n\}$。

生成器是一个小的MLM，会预测输入中`[MASK]`对应的字符，公式如下所示：

$$
p_G(x_t|\mathbf x) = \frac{exp(e(x_t)^Th_G(\mathbf{x})_t)}{\sum_{x'}exp(e(x')^Th_G(\mathbf x)_t)}
$$

其中$e(x_t)$表示token embedding.

可以看到原句中的the和cooked都被遮掩掉了，生成器预测第一个[MASK]是the，第二个是ate，这时判别器再继续判断生成器输出的序列中每个字符是否是被替换字符，其公式如下：

$$
D(\mathbf x, t) = sigmoid(w^Th_D(\mathbf x)_t)
$$

就是加个线性层，然后通过sigmoid激活函数来做二分类。

### 损失函数

这种结构很想GAN，但是实际上，这里不是做对抗学习，而是基于最大似然估计来训练。

将输入序列$\mathbf x = \{x_1, ..., x_n\}$按照15%的概率随机遮掩，被遮掩的token序列为$\mathbf m = \{m_1, ..., m_k\}$，这时

![image](../images/electra_loss.png)

定义如下损失函数：

![image](../images/electra_loss_1.png)

最后将二者结合，即可得到最终的损失函数：

![image](../images/electra_loss_2.png)

### 实验结果

### 总结

预训练模型，预训练有关的实验，其实距离我们很遥远，只能远观而不可亵玩焉，只能看微调的时候它的强大了。

ELECTRA的模型原理很简单，但是效果非常好。

损失函数这里还是有点不清晰。

[^rtd]: replaced token detection